# 2023-12-03

In [1]:
from aocd.models import Puzzle


## Initial solution
Woke up att 5:50am (~Midnight UTC-5) and did it on time. 
```
      --------Part 1--------   --------Part 2--------
Day       Time   Rank  Score       Time   Rank  Score
  3   01:57:43  11730      0   02:14:18   9700      0
```

### Puzzle 1 and 2
Part 1 also took me way too long. I quickly settled on the sliding window approach. I realised a bit too late that, as often with sliding window techniques, you need to consider padding or out of index events. I also realised quite late that retrieving the whole number from inside the sliding window function would be a bit of a pain. Backtracking or going forward to find the whole number seemed overly complex. Instead I created a hash table of all the numbers in the grid. Thus, each index where there was a digit lead to the digits corresponding number. This made the sliding window function much simpler as it only had to fetch the numbers from the hash table.

Generally my approach does the following:
- Interpreted the task as requiring a sliding window approach.

- Created a hashtable where each digit's index in the grid corresponds to the full number, ensuring O(n) complexity.

- Added padding to the grid with '.' to prevent index errors when using the sliding window.

- When the sliding window's central element hits a symbol, searched the window for numbers.

- Used the hashtable index to retrieve the full number associated with each digit.

- Implemented unique IDs for each number to differentiate between them in the window and prevent duplicates.

- Calculated the sum of these unique numbers.

For part two we only had to make minor changes, we stop only at gears `*` and then create a temporary list of the parts in the window. If there are more than two parts in the window we then perform the product operation and add it back to the list that we sum later.

### What do I think of my work?
Im fucking slow. The approach makes sense but I was slow. Keen to see how others have done it.


In [325]:
import itertools
import math

from uuid import uuid4


puzzle = Puzzle(2023, 3)


def find_numbers(grid):
    numbers = {}
    current_number = ""
    current_id = uuid4()

    def add_number_to_table():
        for i in range(len(current_number)):
            numbers.update({(row, col - i - 1): (int(current_number), current_id)})

    for row in range(len(grid)):
        for col in range(len(grid[0])):
            if grid[row][col].isnumeric():
                current_number += grid[row][col]
            else:
                add_number_to_table()
                current_number = ""
                current_id = uuid4()
        if current_number:
            add_number_to_table()

    return numbers


def check_window(
    grid: list[list[str]],
    numbers_table: dict,
    relevant_part_ids: list,
    relevant_part_numbers: list,
    solve_for: str,
    size=1,
    row_pointer=0,
    column_pointer=0,
):
    def add_padding_y(grid, padding):
        return ["." * len(grid[0])] * padding + grid + ["." * len(grid[0])] * padding

    def add_padding_x(grid, padding):
        return [("." * padding) + row + ("." * padding) for row in grid]

    grid = add_padding_y(grid, size)
    grid = add_padding_x(grid, size)

    relevant_rows = grid[
        max(row_pointer - size + size, 0) : min(
            row_pointer + size + size + 1, len(grid)
        )
    ]
    relevant_columns = [
        row[
            max(column_pointer - size + size, 0) : min(
                column_pointer + size + size + 1, len(row)
            )
        ]
        for row in relevant_rows
    ]
    if solve_for == "p1":
        if not relevant_columns[1][1].isnumeric() and relevant_columns[1][1] != ".":
            for x, y in itertools.product(
                range(len(relevant_columns)), range(len(relevant_columns[0]))
            ):
                if relevant_columns[x][y].isnumeric():
                    offset_x, offset_y = x - size, y - size
                    part_number, part_id = numbers_table[
                        (row_pointer + offset_x, column_pointer + offset_y)
                    ]
                    if part_id not in relevant_part_ids:
                        relevant_part_ids.append(part_id)
                        relevant_part_numbers.append(part_number)

    else:
        if not relevant_columns[1][1].isnumeric() and relevant_columns[1][1] == "*":
            neighbour_part_numbers, neighbour_part_ids = [], []
            for x, y in itertools.product(
                range(len(relevant_columns)), range(len(relevant_columns[0]))
            ):
                if relevant_columns[x][y].isnumeric():
                    offset_x, offset_y = x - size, y - size
                    part_number, part_id = numbers_table[
                        (row_pointer + offset_x, column_pointer + offset_y)
                    ]
                    if part_id not in neighbour_part_ids:
                        neighbour_part_ids.append(part_id)
                        neighbour_part_numbers.append(part_number)
            if len(set(neighbour_part_ids)) >= 2:
                relevant_part_ids += neighbour_part_ids
                relevant_part_numbers.append(math.prod(neighbour_part_numbers))
    return relevant_part_ids, relevant_part_numbers


def slide_window(data, solve_for):
    numbers_table = find_numbers(data)
    relevant_part_ids, relevant_part_numbers = [], []
    for row_pointer, column_pointer in itertools.product(
        range(len(data)), range(len(data[0]))
    ):
        relevant_part_ids, relevant_part_numbers = check_window(
            data,
            numbers_table,
            relevant_part_ids,
            relevant_part_numbers,
            solve_for,
            1,
            row_pointer,
            column_pointer,
        )
    return sum(relevant_part_numbers)


puzzle.answer_a = slide_window(puzzle.input_data.splitlines(), "p1")
puzzle.answer_b = slide_window(puzzle.input_data.splitlines(), "p2")
